In [ ]:
print('Start')
from IPython.display import clear_output
import requests
import datetime
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
#Библиотека содержит
import get_data_dic
import distr
from tqdm import tqdm
print('Packages downloaded')
#Вывод на экран текущего времени (час, мин)
localtime = time.localtime(time.time())
print ("Local current time :", localtime[3:5])
#Ссылкы на первую страницу выдачи при поиске
#квартир из определенного района при поиске квартир из ЦАО
def split_link(link):
    return link.split('/')[-2]
time_list =[]

mln_boarders=[]
mln_boarders=[]
for mln_price in range(4,500,1):
    mln_boarders.append(mln_price*0.1e6)
    #print(mln_price)
    if mln_price%5==0:
        #print('eba')
        mln_boarders.append( (mln_price*0.1e6) + 1 )
parsed_files_list_names =[ 'data29_01_2017__21_29_12ADD.csv',
                   'data30_01_2017__00_04_43ADD.csv',
                         'FINAL_data30_01_2017__06_33_11ADD.csv',
                          'data30_01_2017__14_35_18ADD.csv',
                         'data30_01_2017__21_36_42ADD.csv',
                         'data31_01_2017__19_17_49ADD.csv',
                         'data01_02_2017__06_18_27ADD.csv']
print('Already parsed files downloaded')
def add_path(filename):
    big_files_path = 'output/big_files/'
    return big_files_path + filename
parsed_files_list = list(map(add_path,parsed_files_list_names))
print(parsed_files_list)

parsed_df = []
for parsed_file in parsed_files_list:
    df1 = pd.read_csv(parsed_file,sep=';',encoding='cp1251')
    if len(parsed_df)!=0:
        parsed_df = pd.concat([parsed_df,df1])
    else:
        parsed_df = df1.copy()
parsed_links = list(parsed_df['Link'])
del parsed_df
#print(parsed_links)
parsed_links = list(map(split_link,parsed_links))
#print(parsed_links[0:10])
dist_list =[]
dist_list_show=[]
min_boarder=0
for boarder in mln_boarders:
    a = 'https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2'
    b = '&maxprice=' + str(int(boarder-1))
    c = '&minprice=' + str(int(min_boarder))
    d = '&offer_type=flat&'
    e = 'p={}'
    f = '&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
    dist_list_show.append(' from ' + str((min_boarder/1000000)) + ' to ' + str((boarder/1000000))+' mln')
    min_boarder = boarder
    abcdef = a + b + c + d + e + f
    #print(abcdef)
    dist_list.append(abcdef)
#print(dist_list)
#dist_list=[cent]
#X - будущий большой датасет со всеми квартирами и фичами
N=30 #число страниц, по которым мы бегаем, ища
X=[]#all data
#Цикл по всем районам. Номер района в цикле это и есть номер,
#под которым этот районом будет закодирован в конечном датасете
ID=0 #номер квартиры в датасете
print('Start parsing')
MAX_STACK = 0
for distN in tqdm(range( len(dist_list) )):
    print('Search_group ',dist_list_show[distN])
    time1 = time.localtime(time.time())
    #print ("Local current time :", time1[3:5])
    #по главной ссылке находим номера страниц каждой из квартир
    #например, на нулевой итерации links - список уникальных номеров всех
    #квартир из ЦАО
    links=distr.links( dist_list[distN],N )
    #пробегаем во всем квартирам в округе
    print(len(links),end=' - > ')
    filt_links = []
    for link in links:
        if (str(link) ) not in parsed_links:
            filt_links.append(link)
    if len(filt_links) >MAX_STACK:
        MAX_STACK = len(filt_links)
    print(len(filt_links))
    link_loop_couter = 0
    for link in filt_links:
        
        if (link_loop_couter%30==0)|(link_loop_couter==10):
            print(link_loop_couter+1,') ', time.strftime("%H_%M_%S"),end= ' ')
        link_loop_couter+=1
        #словарь из фич для данной квартиры
        ind_data_dic=get_data_dic.get_data_dic (link)
        #добавляем поле District, которое зависит от района
        ind_data_dic['Search_group']=dist_list_show[distN]
        ind_data_dic['ID']=ID
        ind_data_dic['Link']='http://www.cian.ru/sale/flat/' + str(link) + '/'
        ind_data_dic['Global_link'] = dist_list[distN]
        ind_data_dic['parsing_date'] = time.strftime("%d_%m_%Y")
        ID+=1
        #добавляем словарь фич для квартиры в итоговый датасет    
        X.append(ind_data_dic)
    #print("preparing data")
    data=pd.DataFrame(X)
    time_list.append(datetime.datetime.utcnow())
    
    #print(str(distN) + '/' + str(len(dist_list)))
    
    data.to_csv("output\data" +time.strftime("%d_%m_%Y")+'__'+ time.strftime("%H_%M_%S") +'ADD' +".csv",sep=';')
    clear_output()
    #plt.show()
    
data.to_csv("output\FINAL_data" +time.strftime("%d_%m_%Y")+'__'+ time.strftime("%H_%M_%S") +'ADD'+ ".csv",sep=';')
print('Done')

  2%|█▍                                                                             | 11/595 [01:19<1:17:02,  7.91s/it]

Search_group   from 1.2 to 1.3 mln
page  1 https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=1299999&minprice=1200000&offer_type=flat&p=&p=1&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1 page  2 https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=1299999&minprice=1200000&offer_type=flat&p=&p=2&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1 page  3 https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=1299999&minprice=1200000&offer_type=flat&p=&p=3&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1 

In [14]:
dist_list

['https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=399999&minprice=0&offer_type=flat&p={}&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1',
 'https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=499999&minprice=400000&offer_type=flat&p={}&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1',
 'https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=500000&minprice=500000&offer_type=flat&p={}&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1',
 'https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=599999&minprice=500001&offer_type=flat&p={}&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1',
 'https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=699999&minprice=600000&offer_type=flat&p={}&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1',


In [9]:
for Ndist in range(1000):
    if (Ndist<10) | (Ndist<100)&(Ndist%10==0)|(Ndist%50==0):
        print(Ndist,end=' ')
        

0 1 2 3 4 5 6 7 8 9 10 20 30 40 50 60 70 80 90 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 

In [12]:
0.1e6*10

1000000.0

In [ ]:


for i in range(3):
    print(i,end=' ')
    time_list.append(datetime.datetime.utcnow())
    progress_list.append(i)
    time.sleep(1)
import datetime

datetime.datetime.utcnow()

plt.plot(time_list,progress_list)
plt.show()

In [10]:
time.strftime

TypeError: strftime() takes at least 1 argument (0 given)

In [1]:
def heat_map(x,y):
    import numpy as np
    import matplotlib.pyplot as plt
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=200)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    plt.clf()
    #plt.figure(figsize=(10,5))
    plt.imshow(heatmap.T, extent=extent, origin='lower')
    plt.show()
